# Benchmarking Different Quantum Hardware Devices using Grover's Algorithm

## Code Steps:
1. [Libraries, Parameters and Global Variables](#variables)
2. [Constructing Grover's Circuit using Qiskit](#grover_circuit)
3. [Running Grover's Circuits on IBM Hardware](#ibm)
4. [Running Grover's Circuits on AWS Braket](#aws_circuits)
    1. [Running Grover's Circuit on IonQ (Forte)](#ionq)
    2. [Running Grover's Circuit on IQM (Garnet)](#iqm)
5. [Collecting Results from Queued Jobs/Tasks](#results)
5. [Saving Data to CSV and JSON files](#save_data)

## 1. Libraries, Parameters and Global Variables <a name="variables"></a>

In [1]:
# Built-in modules
import numpy as np
import pandas as pd

# Imports from Braket
from qiskit_braket_provider import BraketProvider

# Imports from Qiskit
from qiskit import qasm3

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService

# Import helper functions
from helper import generate_short_key


In [2]:
## Initialize global variables to store input parameters and output

# Initialize the DataFrame for input parameters
param_df = pd.DataFrame(columns=['key', 'target_states', 'quantum_circuit', 'num_qubits', 'depth'])

# Initialize the DataFrame for storing output
output_df = pd.DataFrame(columns=['key', 'device_name', 'transpiled_depth', 'job_id', 'counts', 'execution_time'])

print(param_df)
print(output_df)

# Sample key generation
key = generate_short_key()
print(f"\nGenerated Key: {key}")

Empty DataFrame
Columns: [key, target_states, quantum_circuit, num_qubits, depth]
Index: []
Empty DataFrame
Columns: [key, device_name, transpiled_depth, job_id, counts, execution_time]
Index: []

Generated Key: a2994f6b


In [ ]:
# Read Data from JSON files to DataFrames

param_json = 'saved_data/parameters_20241208_0844.json'
output_json = 'saved_data/output_20241208_0844.json'

param_df = pd.read_json(param_json, orient="records")
param_df['quantum_circuit'] = param_df['quantum_circuit'].apply(lambda qasm: qasm3.loads(qasm))

output_df = pd.read_json(output_json, orient="records")

In [4]:
param_df

,key,target_states,quantum_circuit,num_qubits,depth
0,a9f638fd,[01],"((Instruction(name='h', num_qubits=1, num_clbi...",2,3


In [5]:
output_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,a9f638fd,aer_sim,6,,{'01': 1024},1970-01-01 00:00:00.591352701
1,a9f638fd,aer_noisy,12,,"{'01': 965, '11': 32, '00': 21, '10': 6}",1970-01-01 00:00:13.188663006
2,a9f638fd,ibm_fez,8,cxawmv3rkac00089qk40,,NaT


## 2. Constructing Grover's Circuit using Qiskit <a name="grover_circuit"></a>
1. This section takes an upper limit to the number of qubits $(n_{max})$ in a circuit, describing the size of the database. 
2. A random string of n-bit `target_states`, i.e., states to be searched in the database, is generated to construct the Grover's Oracle. To reduce circuit depth, we set the number of possible target_states to be atmost 4.
3. A total of $n_{max}$ Grover's circuits are generated, each with increasing number of qubits from 2 to $n_{max}$.
4. This list of target states and Grover's circuits are stored in the `param_df` DataFrame.

In [6]:
from helper import generate_random_n_bit_strings

# Maximum number of qubits
n_max = 2

# Generate random target_states for the Oracle
target_states = []

for n in range(2, n_max+1):
    states = generate_random_n_bit_strings(n)
    target_states.append(states)
print(f"Target states: \n {target_states}")


Target states: 
 [['00']]


In [7]:
from grover import grover_circuit

# Generate n_max Grover's circuits and storing in Data Frame
idx = 0
param_df.drop(param_df.index, inplace=True)

for states in target_states:
    circuit = grover_circuit(states)
    key = generate_short_key()
    param_df.loc[idx] = [key, list(states), circuit, circuit.num_qubits, circuit.depth()]
    idx = idx + 1

param_df

,key,target_states,quantum_circuit,num_qubits,depth
0,ea107f17,[00],"((Instruction(name='h', num_qubits=1, num_clbi...",2,3


In [8]:
# Print the Grover's circuits
for qc in param_df["quantum_circuit"]:
    print(qc)

        ┌───┐┌────┐ ░ ┌─┐   
   q_0: ┤ H ├┤0   ├─░─┤M├───
        ├───┤│  Q │ ░ └╥┘┌─┐
   q_1: ┤ H ├┤1   ├─░──╫─┤M├
        └───┘└────┘ ░  ║ └╥┘
meas: 2/═══════════════╩══╩═
                       0  1 


## 3. Running Grover's Circuits on IBM Hardware <a name="ibm"></a>

In [ ]:
# Save an IBM Quantum account as the default account, and load saved credentials
from ibm_qiskit.config import api_key 

QiskitRuntimeService.save_account(
    channel="ibm_quantum", token=api_key, set_as_default=True, overwrite=True
)
service = QiskitRuntimeService()
# service.backends()  # List all available backends

In [ ]:
from ibm_qiskit.run import aer_without_noise, aer_noisy

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# Run circuits on Aer simulator (without noise)
output = aer_without_noise(key_circuit)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aer_sim', depth, '', counts, time]
    

# Select an IBM backend device to model noise
backend = service.backend("ibm_fez")    # Choosing 'ibm_fez' backend 
print(f"Backend Selected: {backend}")

# Run circuits on noisy Aer simulator
output = aer_noisy(key_circuit, backend)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aer_noisy', depth, '', counts, time]

In [ ]:
# Select an IBM backend device

backends = [service.backend("ibm_fez")] 
print(f"Backends Selected: {backends}")

In [ ]:
from ibm_qiskit.run import run_backend_job

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# Run circuits on actual IBM backends
for backend in backends:
    ibm_device = backend.name
    output = run_backend_job(key_circuit, backend)
    for key, job_id, depth in output:
        output_df.loc[len(output_df)] = [key, ibm_device, depth, job_id, '', '']


In [ ]:
# Print all output entries for IBM Hardware
ibm_df = output_df[output_df['device_name'].str.startswith(('aer', 'ibm'))]
ibm_df

## 4. Running Grover's Circuits on AWS Braket <a name="aws_circuits"></a>

In [9]:

# Use Braket SDK Cost Tracking to estimate the cost to run this task
from braket.tracking import Tracker

t = Tracker().start()

In [10]:
# Get AWS provider names
provider = BraketProvider()
print(provider.backends())

[BraketBackend[Ankaa-2], BraketBackend[Aria 1], BraketBackend[Aria 2], BraketBackend[Forte 1], BraketBackend[Garnet], BraketBackend[SV1], BraketBackend[dm1]]


In [ ]:
from aws_braket.run import aws_local_simulator, aws_online_simulator

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# Run circuits on AWS local simulator (zero noise model)
output = aws_local_simulator(key_circuit)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aws_local', depth, '', counts, time]


# Run circuits on noisy AWS StateVector simulator
sim = "SV1"
output = aws_online_simulator(key_circuit, sim)
for key, depth, counts, time in output:
    device = 'aws'+sim
    output_df.loc[len(output_df)] = [key, device, depth, '', counts, time]


c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:500: UserWarning: Device does not support global phase; global phase of 3.141592653589793 will not be included in Braket circuit
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:500: UserWarning: Device does not support global phase; global phase of 3.141592653589793 will not

### A. Running Grover's circuits on IonQ (Forte) <a name="ionq"></a>

In [ ]:
from aws_braket.run import aws_run_task

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))
ionq_device = "Forte 1"
shots = 10

# Run circuits on AWS linked IonQ hardware
output = aws_run_task(key_circuit, ionq_device, shots)
for key, depth, task_arn in output:
    device = 'aws_'+ionq_device
    output_df.loc[len(output_df)] = [key, device, depth, task_arn, '', '']


c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:500: UserWarning: Device does not support global phase; global phase of 3.141592653589793 will not be included in Braket circuit
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:500: UserWarning: Device does not support global phase; global phase of 3.141592653589793 will not be included in Braket circuit
  warnings.warn(


In [ ]:
device
print(device[4:])

Forte 1


### B. Running Grover's circuits on IQM (Garnet) <a name="iqm"></a>

In [ ]:
from aws_braket.run import aws_run_task

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))
iqm_device = "Garnet"
shots = 100

# Run circuits on AWS linked RIgetti hardware
output = aws_run_task(key_circuit, iqm_device, shots)
for key, depth, task_arn in output:
    device = 'aws_'+iqm_device
    output_df.loc[len(output_df)] = [key, device, depth, '', '', '']


In [52]:
# Print all output entries for AWS linked hardware
aws_df = output_df[output_df['device_name'].str.startswith(('aws'))]
aws_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
3,ea107f17,aws_local,3,,{'00': 1024},0.531274
4,ea107f17,aws_sv1,12,,{'00': 1024},12.673005
5,ea107f17,aws_ionq,14,arn:aws:braket:us-east-1:615299756142:quantum-...,,
6,ea107f17,aws_SV1,14,arn:aws:braket:us-west-1:615299756142:quantum-...,,


In [20]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing "
    "unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated "
    "charges do not factor in any discounts or credits, and you may experience additional charges "
    "based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2)."
)
print(
    f"Estimated cost to run simulator tasks: {t.simulator_tasks_cost():.3f} USD"
)
print(
    f"Estimated cost to run the entire task: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.3f} USD"
)

Quantum Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 1024, 'tasks': {'COMPLETED': 1}, 'execution_duration': datetime.timedelta(microseconds=28000), 'billed_execution_duration': datetime.timedelta(seconds=3)}, 'arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1': {'shots': 10, 'tasks': {'CREATED': 1}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run simulator tasks: 0.004 USD
Estimated cost to run the entire task: 1.104 USD


## 5. Collecting Results from Queued Jobs/Tasks <a name="results"></a>


In [21]:
# Print all output entries
output_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,a9f638fd,aer_sim,6,,{'01': 1024},1970-01-01 00:00:00.591352701
1,a9f638fd,aer_noisy,12,,"{'01': 965, '11': 32, '00': 21, '10': 6}",1970-01-01 00:00:13.188663006
2,a9f638fd,ibm_fez,8,cxawmv3rkac00089qk40,,NaT
3,ea107f17,aws_local,3,,{'00': 1024},0.531274
4,ea107f17,aws_sv1,12,,{'00': 1024},12.673005
5,ea107f17,aws_ionq,14,arn:aws:braket:us-east-1:615299756142:quantum-...,,


In [ ]:
# Get results from real IBM backend
from ibm_qiskit.helper import update_dataframe_with_ibm_results

service = QiskitRuntimeService()
update_dataframe_with_ibm_results(service, output_df)

# Print results from IBM devices
ibm_df = output_df[output_df['device_name'].str.startswith(('aer', 'ibm'))]
ibm_df

Job cxawmv3rkac00089qk40 is not completed yet. Current status: ERROR


,key,device_name,transpiled_depth,job_id,counts,execution_time
0,a9f638fd,aer_sim,6,,{'01': 1024},1970-01-01 00:00:00.591352701
1,a9f638fd,aer_noisy,12,,"{'01': 965, '11': 32, '00': 21, '10': 6}",1970-01-01 00:00:13.188663006
2,a9f638fd,ibm_fez,8,cxawmv3rkac00089qk40,,NaT


In [82]:
# Get results from AWS linked devices
from aws_braket.helper import update_dataframe_with_aws_results

update_dataframe_with_aws_results(output_df)

# Print results from AWS linked devices
aws_df = output_df[output_df['device_name'].str.startswith(('aws'))]
aws_df

Updated job arn:aws:braket:us-west-1:615299756142:quantum-task/38f2cb3b-d9a1-478b-a2fe-c4fdeada972d: Counts = {'00': 1024}


,key,device_name,transpiled_depth,job_id,counts,execution_time
3,ea107f17,aws_local,3,,{'00': 1024},0.531274
6,ea107f17,aws_SV1,14,arn:aws:braket:us-west-1:615299756142:quantum-...,{'00': 1024},


## 6. Saving Data to CSV and JSON files<a name="save_data"></a>

In [ ]:
# Creating Timestamps (Run only once for a given set of data)
import os
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M')

subfolder = 'saved_data'

# Create the subfolder if it does not exist
os.makedirs(subfolder, exist_ok=True)

# Create file names
param_csv = os.path.join(subfolder, f"parameters_{timestamp}.csv")
output_csv = os.path.join(subfolder, f"output_{timestamp}.csv")

param_json = os.path.join(subfolder, f"parameters_{timestamp}.json")
output_json = os.path.join(subfolder, f"output_{timestamp}.json")

In [ ]:
# Save DataFrames to CSV files

# # Give file name to update existing files
# param_csv = 'saved_data/parameters_20241208_0844.csv'
# output_csv = 'saved_data/output_20241208_0844.csv'

param_df.to_csv(param_csv, index=False)  # index=False prevents extra index column
output_df.to_csv(output_csv, index=False)

In [ ]:
# Save DataFrames to JSON files

# # Give file name to update existing files
# param_json = 'saved_data/parameters_20241208_0844.json'
# output_json = 'saved_data/output_20241208_0844.json'

param_df['quantum_circuit'] = param_df['quantum_circuit'].apply(lambda qc: qasm3.dumps(qc))
param_df.to_json(param_json, orient='records')
param_df['quantum_circuit'] = param_df['quantum_circuit'].apply(lambda qc: qasm3.loads(qc))

output_df.to_json(output_json, orient='records')

----------------------------- End of Notebook ------------------------------